In [1]:
## Reading Files
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import sklearn
import plotly.express as px
from pandas import Period
from scipy import stats
import re

In [3]:
ameco_1623 = pd.read_csv('/content/drive/MyDrive/Capstone Data/ameco_data.csv',low_memory=False)
df = ameco_1623.copy()

In [4]:
df = df[['X.','Customer.No.','Customer.Name','Customer.Category',
                        'Ship.To.State','Invoice.Date','Strikeforce.Flag','Stock.Status',
                        'Item.No.','Item.Group','Product.Classification',
                        'UoM.Code','Quantity','Inventory.Cost','Price',
                        'DPM.Factor','DPM.Price']]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155850 entries, 0 to 1155849
Data columns (total 17 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   X.                      1155850 non-null  object 
 1   Customer.No.            1155850 non-null  object 
 2   Customer.Name           1155835 non-null  object 
 3   Customer.Category       590461 non-null   object 
 4   Ship.To.State           571691 non-null   object 
 5   Invoice.Date            1155850 non-null  object 
 6   Strikeforce.Flag        572301 non-null   object 
 7   Stock.Status            691242 non-null   object 
 8   Item.No.                1155840 non-null  object 
 9   Item.Group              572291 non-null   object 
 10  Product.Classification  572290 non-null   object 
 11  UoM.Code                1155835 non-null  object 
 12  Quantity                1155850 non-null  object 
 13  Inventory.Cost          1155847 non-null  object 
 14  Pr

In [6]:
numerical_columns = ['Inventory.Cost','Price','Quantity']

In [7]:
df[numerical_columns] = df[numerical_columns].replace({',': ''}, regex=True).astype(float)

In [8]:
df = df[df['Price'] > 0]

## Dataset Clean

### Customer Category Remap

#### Make the cusotmer no consistent

In [9]:
df['Customer.Name'] = df['Customer.Name'].str.replace(r'[^\w\s]+', '')

<ipython-input-9-ca9b5d3313d0>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Customer.Name'] = df['Customer.Name'].str.replace(r'[^\w\s]+', '')


In [10]:
df['Customer.Name'] = df['Customer.Name'].map(lambda x: x.strip() if isinstance(x, str) else x)

In [11]:
df['Customer.Name']

0                             BEN HUR
1                 CUSTOM CONVEYOR INC
2                 CUSTOM CONVEYOR INC
3                 CUSTOM CONVEYOR INC
18                       KENNEDY TANK
                      ...            
1155845    ADS MANUFACTURING OHIO LLC
1155846    ADS MANUFACTURING OHIO LLC
1155847    ADS MANUFACTURING OHIO LLC
1155848    ADS MANUFACTURING OHIO LLC
1155849    ADS MANUFACTURING OHIO LLC
Name: Customer.Name, Length: 1134769, dtype: object

In [12]:
def map_customer_no(row):
    if len(row['Customer.No.']) == 6 and 'PRE' in row['X.']:
        return 'C' + row['Customer.No.']
    elif len(row['Customer.No.']) == 5 and 'PRE' in row['X.']:
        return 'C0' + row['Customer.No.']
    elif len(row['Customer.No.']) == 4 and 'PRE' in row['X.']:
        return 'C00' + row['Customer.No.']
    elif len(row['Customer.No.']) == 3 and 'PRE' in row['X.']:
        return 'C000' + row['Customer.No.']
    elif len(row['Customer.No.']) == 2 and 'PRE' in row['X.']:
        return 'C0000' + row['Customer.No.']
    elif len(row['Customer.No.']) == 1 and 'PRE' in row['X.']:
        return 'C00000' + row['Customer.No.']
    else:
        return row['Customer.No.']

In [13]:
df['Customer.No.'] = df.apply(map_customer_no, axis=1)

In [14]:
print(f"The number of unique customer numberis now {df['Customer.No.'].nunique()}.")
print(f"The number of unique customer name is {df['Customer.Name'].nunique()}.")

The number of unique customer numberis now 2084.
The number of unique customer name is 2123.


In [15]:
customer_pair = df[['Customer.No.','Customer.Name']]
customer_pair = customer_pair.groupby('Customer.No.')['Customer.Name'].nunique().reset_index().rename(columns={'Customer.Name':'Count'})

In [16]:
customer_pair[customer_pair['Count']!=1]

,Customer.No.,Count
13,C000109,2
45,C000449,2
63,C000612,2
78,C000767,2
109,C001072,2
...,...,...
1681,C109882,2
1683,C109887,2
1692,C109902,2
1694,C109905,2


In [17]:
df[df['Customer.No.']=='C001072'].sample(5)

,X.,Customer.No.,Customer.Name,Customer.Category,Ship.To.State,Invoice.Date,Strikeforce.Flag,Stock.Status,Item.No.,Item.Group,Product.Classification,UoM.Code,Quantity,Inventory.Cost,Price,DPM.Factor,DPM.Price
21061,PRE 21062,C001072,APPLIED INDUSTRIAL TECHNOLOGIE,P1,NaN,1/1/2016,NaN,N,ENE M783055,NaN,NaN,EA,1.0,3.620,6.00,NaN,NaN
21056,PRE 21057,C001072,APPLIED INDUSTRIAL TECHNOLOGIE,P1,NaN,1/1/2016,NaN,N,ENE CA561024,NaN,NaN,EA,1.0,5.920,9.80,NaN,NaN
365277,PRE 365278,C001072,APPLIED INDUSTRIAL TECHNOLOGIE,P1,NaN,1/1/2018,NaN,N,ENE IPL100-SEAK,NaN,NaN,EA,1.0,61.776,132.00,NaN,NaN
21058,PRE 21059,C001072,APPLIED INDUSTRIAL TECHNOLOGIE,P1,NaN,1/1/2016,NaN,N,ENE Y252167,NaN,NaN,EA,1.0,24.900,41.20,NaN,NaN
858570,SAP 275022,C001072,APPLIED INDUSTRIAL TECHNOLOGIES,NaN,KY,6/9/2023,N,Non Stock,VIC 0781-3600,Const Supplies CS/SD,EQUMIS,EA,1.0,202.340,256.95,0.0,0


#### read customer category

In [18]:
customer_df = pd.read_excel('/content/drive/MyDrive/Capstone Data/20240312 Customer Price List Assignment.xlsx',usecols=[0,1,2,3,4]).copy()

In [19]:
customer_df.sample(10)

,#,Customer No.,Customer Name,DPM Exempt,Price List Name
2068,2069,C109374,INACTIVE DESERT BOILERS & CONTROLS INC.,N,Good
778,779,C103152,KONE ELEVATOR,N,Good
3559,3560,C111214,"A-1 EQUIPMENT SERVICE AND REPAIR, INC.",N,Good
2362,2363,C109714,CHILLRX CRYOTHERAPY,N,Good
3324,3325,C110821,"GUTIER, LTD.",N,Good
72,73,C000604,OTIS NORTH AMERICA HEADQUARTERS,N,Good
212,213,C001588,QUALITY MECHANICAL,N,Good
2887,2888,C110289,DMR Millwright & Industrial Solutions,N,Good
1998,1999,C109286,US PARTS LOCATORS,N,Good
1832,1833,C109089,CONTRACTORS CRANE COMPANY INC,N,Good


In [20]:
customer_df['Customer No.'].nunique()

3595

In [21]:
customer_df[customer_df['Customer No.'] == 'C001072']

,#,Customer No.,Customer Name,DPM Exempt,Price List Name
128,129,C001072,APPLIED INDUSTRIAL TECHNOLOGIES,N,Good


In [22]:
customer_df['Customer No.'].unique()

array(['C000002', 'C000014', 'C000021', ..., 'C111271', 'C111272',
       'C180650'], dtype=object)

In [23]:
no_list_1 = list(customer_df['Customer No.'].unique())
no_list_2 = list(df['Customer.No.'].unique())
print(f'Length of the updated file is {len(no_list_1)}')
print(f'Length of the original file is {len(no_list_2)}')
no_list_2 in no_list_1

Length of the updated file is 3595
Length of the original file is 2084


False

In [24]:
customer_dict = dict(zip(customer_df['Customer No.'],customer_df['Customer Name']))

In [25]:
def map_customer_name(row):
  if row['Customer.No.'] in customer_dict.keys():
    return customer_dict.get(row['Customer.No.'])
  else:
    return row['Customer.Name']

In [26]:
df['customer_name'] = df.apply(map_customer_name, axis=1)

#### Map the customer category

In [27]:
category_dict = dict(zip(customer_df['Customer No.'],customer_df['Price List Name']))

In [28]:
def map_category(row):
  if row['Customer.No.'] in category_dict.keys():
    return category_dict.get(row['Customer.No.'])
  else:
    return row['Customer.Category']

In [29]:
df['customer_category'] = df.apply(map_category, axis=1)

In [30]:
df['customer_category'].value_counts()

Good            611198
Better          333080
Best            170152
Average Cost     17723
List Price        1399
M1                 464
P4                 290
P1                 268
P3                  90
P2                  66
M4                  29
P5                  10
Name: customer_category, dtype: int64

Check whether the customer belongs to a category consistently

> Unfortunately there are about 1507 has inconsistent customer categories.

In [31]:
df[(df['Customer.Category'] != df['customer_category'])&(~df['Customer.Category'].isnull())]['Customer.No.'].nunique()

1507

#### Mark AMECO sub companies

In [32]:
def mark_own_company(row):
  if row['customer_name'] in [
    "F&M MAFCO, INC",
    "CHRISTIANSTED EQUIPMENT, LTD.",
    "F&M MAFCO LLC",
    "EQUIPMENTSHARE.COM INC",
    "AMECO IC",
    "AMECO CANADA IC",
    "AMECO IC - VISTRA",
    "AMECO SERVICES INC (TRANSMOUNTAIN)"]:
    return True
  else:
    return False


In [33]:
df['Own'] = df.apply(mark_own_company,axis=1)

In [34]:
df['Own'].value_counts()

False    1102450
True       32319
Name: Own, dtype: int64

### Transform Datetime

In [35]:
## date time
df['Invoice.Date'] = pd.to_datetime(df['Invoice.Date'])
df['Year'] = df['Invoice.Date'].dt.year
df['Month'] = df['Invoice.Date'].dt.month
df['Quarter'] = df['Invoice.Date'].dt.to_period('Q')

In [ ]:
def map_quarter(row):
  if row['Year'] == 2016:
    return row['Quarter']
  elif row ['Year'] == 2020:
    return Period('2019Q1', freq='Q-DEC')
  else:
    return row['Quarter'] - 4
df['Prev_Quarter'] = df.apply(map_quarter, axis=1)
df['Prev_Quarter'] = df['Prev_Quarter'].apply(lambda x: Period(x, freq='Q-DEC'))

### Flag

In [ ]:
## Flag mapping
def stock_flag(row):
    if row['Stock.Status'] == 'S' or row['Stock.Status'] == 'Stock':
        return 'Y'
    elif row['Stock.Status'] == 'N' or row['Stock.Status'] == 'Non Stock':
        return 'N'
    else:
      if row['Strikeforce.Flag'] == 'Y':
        return 'Y'
      elif row['Strikeforce.Flag'] == 'N':
        return 'N'
      else:
        return 'Y'

df['Stock.Flag'] = df.apply(stock_flag, axis=1)

In [ ]:
df.drop(columns={'Strikeforce.Flag','Stock.Flag'},inplace=True)

### Item group & Product Classification

In [ ]:
## Item group and product classification
def fill_missing_most_frequent(group):
    mode_values = group.mode()
    if not mode_values.empty:
        mode_value = mode_values.iloc[0]
        return group.fillna(mode_value)
    else:
        return group

df['Product.Classification'] = df.groupby('Item.No.')['Product.Classification'].transform(fill_missing_most_frequent)
df['Item.Group'] = df.groupby('Item.No.')['Item.Group'].transform(fill_missing_most_frequent)

### Final Dataset

In [ ]:
df.drop(columns=['X.','Customer.Name'],inplace=True)

In [ ]:
#df.to_csv('/content/drive/MyDrive/Capstone Data/updated_customer_dataset.csv',index=False)

In [ ]:
df

## Target Profit Margin Dataset

In [ ]:
tg_pm_df = pd.read_csv('/content/drive/MyDrive/Capstone Data/target_margin.csv',skiprows=2,usecols=[0,1,2,3,4,17,18,19])

In [ ]:
tg_pm_df = tg_pm_df.rename(columns=lambda x:x.strip())

In [ ]:
tg_pm_df = tg_pm_df.melt(id_vars=['Product Class'],value_vars=['Good','Better','Best'],var_name='Category',value_name='Values')

In [ ]:
tg_pm_df[tg_pm_df['Product Class']=='POWACC']

In [ ]:
target_product_ls = list(tg_pm_df['Product Class'].unique())

We do have some information about 21.22 expected profit margin, but might not be helpful since they have 5 categories.

## Profit Margin Exploration

#### Calculate profit margin for the entire dataset

In [ ]:
df.columns

In [ ]:
profit_df = df[['Customer.No.','customer_name', 'customer_category','Own','Year', 'Month',
       'Quarter', 'Prev_Quarter','Ship.To.State', 'Invoice.Date',
       'Stock.Status', 'Item.No.', 'Item.Group', 'Product.Classification',
       'UoM.Code', 'Quantity', 'Inventory.Cost', 'Price']]

In [ ]:
# profit_df['Actual Margin'] = (profit_df['Price'] * profit_df['Quantity'] - profit_df['Inventory.Cost'] * profit_df['Quantity'])/(profit_df['Price'] * profit_df['Quantity'])*100
profit_df['Actual Margin'] = (profit_df['Price'] - profit_df['Inventory.Cost'])/(profit_df['Price'])*100

In [ ]:
# profit_df = profit_df.query('Own == False')
ex_own_profit_df = profit_df[profit_df['Own'] == False]

In [ ]:
ex_own_profit_df

#### Profit 2020-2023 Dataset Calculations

#### Profit 2022 Dataset

In [ ]:
ex_own_profit_df_22 = ex_own_profit_df[ex_own_profit_df['Year']==2022]

In [ ]:
ex_own_profit_df_22['customer_category'].value_counts()

In [ ]:
ex_own_profit_df_22 = ex_own_profit_df_22[ex_own_profit_df_22['customer_category'] != 'List Price']

In [ ]:
product_list = list(ex_own_profit_df_22['Product.Classification'].unique())

In [ ]:
print(f'The unique number of products in the original dataset is {len(target_product_ls)}')
print(f'The unique number of products in the target dataset is {len(product_list)}')

In [ ]:
ex_own_profit_df_22['Item.Group'].value_counts()

#### Profit 2023 Dataset

In [ ]:
ex_own_profit_df_23 = ex_own_profit_df[ex_own_profit_df['Year']==2023]

In [ ]:
ex_own_profit_df_23['customer_category'].value_counts()

In [ ]:
ex_own_profit_df_23 = ex_own_profit_df_23[ex_own_profit_df_23['customer_category'] != 'List Price']

In [ ]:
product_list = list(ex_own_profit_df_23['Product.Classification'].unique())

In [ ]:
print(f'The unique number of products in the original dataset is {len(target_product_ls)}')
print(f'The unique number of products in the target dataset is {len(product_list)}')

In [ ]:
ex_own_profit_df_23

In [ ]:
ex_own_profit_df_23['Item.Group'].value_counts()

#### Calculate the proportion of target

### Calculating the profit margin 2023

Here we can have two methods, not sure if that matters or not
but
a) calculate the profit margin for each transaction and then do an average
b) adding up the quantity price cost and then do a calculation

In [ ]:
calc_profit_tb = ex_own_profit_df_23.groupby(['Item.Group','Product.Classification','customer_category',])['Actual Margin','Price'].mean().reset_index()

In [ ]:
calc_profit_tb = calc_profit_tb.rename(columns = {'Product.Classification':'Product Class','customer_category': 'Category'})
calc_profit_tb

In [ ]:
final_df_23 = calc_profit_tb.merge(tg_pm_df, on=['Product Class', 'Category'])
final_df_23

In [ ]:
final_df_23.to_csv('/content/drive/MyDrive/Capstone Data/2023_pm_data.csv',index=False)

In [ ]:
final_df_23['Item.Group'].value_counts()

In [ ]:
final_df_23 = final_df_23.rename(columns={'Values':'Target Margin'})
final_df_23 = final_df_23[final_df_23['Item.Group']!='OPN001']

In [ ]:
final_df_23 = final_df_23.sort_values(['Item.Group','Product Class'])

In [ ]:
final_df_23['Target Margin'] = final_df_23['Target Margin'].str.replace('%', '').astype(float)

In [ ]:
def plot_margin(df, customer_category):
  df = df[df['Category']==customer_category]
  plt.figure(figsize=(10, 6))
  sns.scatterplot(data=df, x=df["Product Class"], y=df['Actual Margin']-df["Target Margin"], hue="Item.Group")
  # Remove tick labels for x-axis
  plt.xticks([])
  plt.ylabel('Difference')
  plt.title(f'The Difference between Expected Profit Margin and Actual Profit Margin in 2023 by\
  Product Class and {customer_category} Customer.')
  plt.axhline(y=0, color="black", linestyle="--")
  for index, row in df.iterrows():
        difference = row['Actual Margin'] - row['Target Margin']
        if difference < -10:  # Define your threshold for large values
            plt.text(row["Product Class"], difference, f"{row['Product Class']}: {difference:.2f}", ha='center', va='bottom', fontsize=8, color='blue')
  plt.show()

In [ ]:
customer_list = final_df_23['Category'].unique()
for each in customer_list:
  plot_margin(final_df_23,each)

Many large differences

In [ ]:
final_df_23_sorted = final_df_23.sort_values(by='Actual Margin', ascending=True)
final_df_23_sorted

In [ ]:
calc_profit_df[(calc_profit_df['Product.Classification'] == 'WLDGAS') & (calc_profit_df['Category'] == 'Good')]

In [ ]:
calc_profit_tb2=calc_profit_df.groupby(['Item.Group','Product.Classification','UoM.Code','customer_category',]).apply(
    lambda row:(100*((row['Price']-row['Inventory.Cost'])*row['Quantity'].sum()))/(row['Price']*row['Quantity'].sum())).reset_index()

In [ ]:
calc_profit_tb2

### Wang jiahua

In [ ]:
#Jiahua Wang Start
product_class = good_customer_df[(good_customer_df['Item.Group'] == 'Welding Equip WS/SD') | (good_customer_df['Item.Group'] == 'Const Supplies CS/SD')]['Product.Classification'].unique()
len(product_class)

In [ ]:
good_customer_df[good_customer_df['Product.Classification'].notna()]

In [ ]:
#select all rows with product class in product_class
good_customer_df_welding = good_customer_df[good_customer_df['Product.Classification'].isin(product_class)]
good_customer_df_welding

In [ ]:
good_customer_df_welding[good_customer_df_welding['Product.Classification'] == 'TIGPAR']

In [ ]:
# for each year, generate profit margin of each product in product_class by calculating sum of (price * quantity) / sum of (inventory.cost * quantity)

annual_profit_margin_product = {}
for year in good_customer_df_welding['Year'].unique():
  year_df = good_customer_df_welding[good_customer_df_welding['Year'] == year]
  annual_profit_margin_product[year] = {}
  for product in product_class:
    product_df = year_df[year_df['Product.Classification'] == product]
    revenue = (product_df['Price'] * product_df['Quantity']).sum()
    cost = (product_df['Inventory.Cost'] * product_df['Quantity']).sum()
    profit_margin = (revenue / cost - 1) * 100
    annual_profit_margin_product[year][product] = profit_margin


In [ ]:
good_customer_df_welding[good_customer_df_welding['Product.Classification'] == 'Welding Equip WS/SD']

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/Capstone Data/margin_final.csv', low_memory = False)
df2

In [ ]:
# prompt: drop UoM.Code column and delete those duplicate rows
df3 = df2.copy()
df3.drop(columns=['UoM.Code','2016','2017','2018','2019','2020','2023'], inplace=True)
df3.drop_duplicates(inplace=True)
df3


In [ ]:
years = [2016,2017,2018,2019,2020,2023]
for year in years:
  df3[year] = df3['ProductClass'].apply(lambda x: annual_profit_margin_product[year][x])
df3

In [ ]:
df3 = df3[(df3['Item.Group'] != 'Rental KO RE/ESG') & (df3['Item.Group'] != 'OPN001')]

In [ ]:
df3.to_csv('profit_margin.csv', index = False)

In [ ]:
# Filter the DataFrame
filtered_df = df3.loc[(df3['Item.Group'] == 'Const Supplies CS/SD') & df3[2023].notna()]

# Check for NaN values and drop them
filtered_df = filtered_df.dropna(subset=['Good', 2023])

# Check for and remove infinite values
filtered_df = filtered_df.replace([np.inf, -np.inf], np.nan).dropna(subset=['Good', 2023])

# Now perform the t-test with the cleaned DataFrame
t_statistic, p_value = stats.ttest_rel(filtered_df['Good'], filtered_df[2023])
print(p_value)

In [ ]:
# Filter the DataFrame
filtered_df = df3.loc[(df3['Item.Group'] == 'Welding Equip WS/SD') & df3[2023].notna()]

# Check for NaN values and drop them
filtered_df = filtered_df.dropna(subset=['Good', 2023])

# Check for and remove infinite values
filtered_df = filtered_df.replace([np.inf, -np.inf], np.nan).dropna(subset=['Good', 2023])

# Now perform the t-test with the cleaned DataFrame
t_statistic, p_value = stats.ttest_rel(filtered_df['Good'], filtered_df[2023])
print(p_value)

In [ ]:
# plot df3['ProductClass'] against df3['Good'] - df3[2023] in a scatterplot and delete tickers for productclass, but use different color for df3[Item.Group] == Const Supplies CS/SD and df3[Item.Group] == Welding Equip WS/SD, also add a horizontal line at 0

import matplotlib.pyplot as plt
import seaborn as sns

# Create a scatterplot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df3, x=df3["ProductClass"], y=df3["Good"] - df3[2023], hue="Item.Group")

# Remove tick labels for x-axis
plt.xticks([])
plt.ylabel('Difference')
plt.title('The Difference between Expected Profit Margin and Actual Profit Margin in 2023 by Product Class and Category')
# Add a horizontal line at y=0
plt.axhline(y=0, color="black", linestyle="--")

# Show the plot
plt.show()


In [ ]:
## Adding inflation
inflation = pd.read_excel('/content/drive/MyDrive/Capstone Data/inflation.xlsx',skiprows=11)
month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
inflation.rename(columns=month_map, inplace=True)
inflation.drop(columns=['HALF1','HALF2'],inplace=True)
melted_inflation = pd.melt(inflation, id_vars=['Year'], var_name='Month', value_name='Inflation')

In [ ]:
## Map Regions
state_to_region = {
    # Pacific
    'CA': 'Pacific', 'WA': 'Pacific', 'OR': 'Pacific', 'AK': 'Pacific', 'HI': 'Pacific',
    # West
    'MT':'West','ND':'West','ID': 'West', 'SD':'West', 'WY': 'West', 'NE':'West','NV':'West','UT': 'West','CO':'West','AZ':'West','NM':'West',
    # Midwest
    'IL': 'Midwest', 'OH': 'Midwest', 'MI': 'Midwest', 'IN': 'Midwest', 'WI': 'Midwest', 'IA': 'Midwest', 'MN': 'Midwest', 'MO': 'Midwest', 'KS': 'Midwest', 'NE': 'Midwest', 'SD': 'Midwest', 'ND': 'Midwest', 'KY':'Midwest', 'WV': 'Midwest',
    # Central
    'KS' : 'Central', 'MO': 'Central', 'TN': 'Central', 'OK' : 'Central', 'AR': 'Central',
    # Gulf Coast
    'TX': 'Gulf_Coast','LA': 'Gulf_Coast', 'MS': 'Gulf_Coast', 'AL': 'Gulf_Coast', 'GA': 'Gulf_Coast', 'FL': 'Gulf_Coast',
    # Mid-Atlantic
    'PA' : 'Mid-Atlantic',  'NJ': 'Mid-Atlantic', 'DE':'Mid-Atlantic', 'MD':'Mid-Atlantic', 'VA':'Mid-Atlantic', 'NC':'Mid-Atlantic', 'SC': 'Mid-Atlantic',
    # Northeast
    'NY': 'Northeast', 'CT': 'Northeast', 'MA': 'Northeast', 'RI': 'Northeast', 'VT': 'Northeast', 'NH': 'Northeast', 'ME': 'Northeast',
    # # Northeast
    # 'NY': 'Northeast', 'NJ': 'Northeast', 'PA': 'Northeast', 'CT': 'Northeast', 'MA': 'Northeast', 'RI': 'Northeast', 'VT': 'Northeast', 'NH': 'Northeast', 'ME': 'Northeast', 'MD': 'Northeast',
    # # Southeast
    # 'DE': 'Southeast', 'WV': 'Southeast', 'VA': 'Southeast', 'NC': 'Southeast', 'SC': 'Southeast', 'GA': 'Southeast', 'FL': 'Southeast', 'DC': 'Southeast', 'KY': 'Southeast','TN':'Southeast','AR': 'Southeast','MS':'Southeast','AL':'Southeast','LA':'Southeast',
    # # East
    # 'NY': 'East', 'NJ': 'East', 'PA': 'East', 'CT': 'East', 'MA': 'East', 'RI': 'East', 'VT': 'East', 'NH': 'East', 'ME': 'East', 'MD': 'East',
    # # Southeast
    # 'DE': 'East', 'WV': 'East', 'VA': 'East', 'NC': 'East', 'SC': 'East', 'GA': 'East', 'FL': 'East', 'DC': 'East', 'KY': 'East','TN':'East','AR': 'East','MS':'East','AL':'East','LA':'East',
    # # Southwest
    # 'TX': 'Southwest', 'AZ': 'Southwest', 'NM': 'Southwest', 'OK': 'Southwest',
    # # Other
    # 'BC':'Other','AB':'Other','QC':'Other'
}

df['Ship.To.State'] = df['Ship.To.State'].replace(state_to_region)

In [ ]:
## Final dataset
merged_df = pd.merge(good_customer_df_welding, melted_inflation, on=['Year', 'Month'], how='left')
columns_to_keep = ['Customer.Name','Customer.Category','Ship.To.State','Item.No.','Item.Group','Product.Classification','UoM.Code','Quantity','Stock.Flag','Inventory.Cost','Price','DPM.Factor','DPM.Price','Invoice.Date','Year','Month','Quarter','Inflation','Prev_Quarter']
df = merged_df[columns_to_keep]

In [ ]:
grouped_df = df.groupby(['Item.No.', 'Quarter'])
mean_prices = grouped_df['Price'].mean().reset_index()
mean_prices_dict = mean_prices.set_index(['Item.No.','Quarter']).to_dict()['Price']
df['prev_quar_price'] = df.apply(lambda row: mean_prices_dict.get((row['Item.No.'],row['Prev_Quarter'])),axis=1)

In [ ]:
final_df = df[(df['UoM.Code'] == 'EA') &
                    (df['Customer.Category'] == 'Good') &
                    ((df['Item.Group'] == 'Const Supplies CS/SD') |
                     (df['Item.Group'] == 'Welding Equip WS/SD')) &
                      (df['Inventory.Cost'] > 0) &
                      (df['Price'] > 0)]
final_df.rename(columns={'DPM.Factor':'Factor','DPM.Price':'DPMPrice','Customer.Name':'Name','Customer.Category': 'Category', 'Item.No.':'Item','UoM.Code':'UoM','Stock.Flag':'Stock','Inventory.Cost':'Cost','Invoice.Date':'Date','Product.Classification':'Classification','Item.Group':'Group','Ship.To.State':'State'}, inplace=True)

In [ ]:
#Build Model
import statsmodels.formula.api as smf

In [ ]:
train_df = final_df[final_df['Year']!=2023]
train_df['log_Price'] = train_df['Price'].apply(lambda x: np.log(x))
train_df['log_Cost'] = train_df['Cost'].apply(lambda x: np.log(x))
train_df['log_prev_quar_price'] = train_df['prev_quar_price'].apply(lambda x: np.log(x))

In [ ]:
formula = 'log_Price ~ C(Group) + C(Stock) + log_Cost + Quantity'
model = smf.ols(formula=formula, data=train_df).fit()

print(model.summary())

In [ ]:

APPFLA_df = good_customer_df_welding[good_customer_df_welding['Product.Classification'] == 'SAFVIS']
plt.scatter(APPFLA_df['Quantity'], APPFLA_df['Price'])

In [ ]:
# Step 1: Calculate Price-to-Cost Ratio
APPFLA_df['Price-to-Cost Ratio'] = APPFLA_df['Price'] / APPFLA_df['Inventory.Cost']

# Step 2: Determine Base Price-to-Cost Ratio (e.g., for the smallest quantity)
base_ratio = APPFLA_df.loc[APPFLA_df['Quantity'] == APPFLA_df['Quantity'].min(), 'Price-to-Cost Ratio'].mean()

# Step 3: Calculate Discount Rate
APPFLA_df['Discount Rate (%)'] = (1 - APPFLA_df['Price-to-Cost Ratio'] / base_ratio) * 100

In [ ]:
# prompt: find average price-to-cost ratio at each quantity existed in APPFLA_df

import pandas as pd

# Calculate the average price-to-cost ratio for each quantity
average_ratios = APPFLA_df.groupby('Quantity')['Price-to-Cost Ratio'].mean()

# Print the average price-to-cost ratios
print(average_ratios)


In [ ]:
plt.plot(average_ratios.index, average_ratios.values)

In [ ]:
tt = APPFLA_df[["Quantity","Inventory.Cost","Price","Price-to-Cost Ratio","Discount Rate (%)"]]
tt[tt['Discount Rate (%)'] > 0]
tt

In [ ]:
APPFLA_df[['Quantity','Discount Rate (%)']]

In [ ]:
# Prepare data
X = APPFLA_df[['Quantity']]  # Independent variable
y = APPFLA_df['Discount Rate (%)']  # Dependent variable

# Optionally split the data - in this simple example, we'll use all data for fitting
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Choose and fit the model
model = LinearRegression()
model.fit(X, y)


In [ ]:
X, y

In [ ]:
#Jiahua Wang End here

In [ ]:
good_customer_df['Year'].value_counts()

In [ ]:
product_class = good_customer_df[good_customer_df['Item.Group'] == 'Welding Equip WS/SD']['Product.Classification'].unique()
for each in product_class:
  if good_customer_df[(good_customer_df['Item.Group'] == 'Welding Equip WS/SD')&(good_customer_df['Product.Classification'] == each)]['UoM.Code'].nunique() > 1:
    print(each)

In [ ]:
result_df = good_customer_df.groupby(['Year','Item.Group','Product.Classification','UoM.Code'])[['Inventory.Cost','Price','Quantity']].sum().reset_index()

In [ ]:
result_df['Profit_Margin'] = (result_df['Price'] - result_df['Inventory.Cost'])/result_df['Inventory.Cost'] * 100

In [ ]:
result_df

In [ ]:
result_df.to_csv('profit_margin_calc.csv',index=False)

In [ ]:
selected_data = result_df[result_df['Product.Classification'] == 'ABRBEL']

plt.figure(figsize=(10, 6))

# Plot Quantity by UoM.Code
sns.scatterplot(data=selected_data, x='Year', y='Quantity', hue='UoM.Code', style='UoM.Code', markers=True, palette='Set2',legend='full')

# Plot Profit Margin by UoM.Code
sns.scatterplot(data=selected_data, x='Year', y='Profit_Margin', hue='UoM.Code', style='UoM.Code', markers=True, palette='Set1',legend='full')

# Set labels and title
plt.xlabel('Year')
plt.ylabel('Value')
plt.title('Quantity and Profit Margin by UoM.Code over Years for ABRBEL')

# Show legend
plt.legend(title='UoM.Code')

# Show the plot
plt.show()


In [ ]:
pilot_df = good_customer_df[(good_customer_df['Item.Group'] == 'Welding Equip WS/SD')&(good_customer_df['Product.Classification'] == 'SAFRES')]

In [ ]:
years = pilot_df['Year'].unique()

for year in years:
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
    year_data = pilot_df[pilot_df['Year'] == year]

    # Plot Profit Margin over Invoice Date
    axes[0].scatter(year_data['Invoice.Date'], year_data['Profit_Margin'], color='blue')
    axes[0].set_title(f'Profit Margin Over Date - Year {year}')
    axes[0].set_xlabel('Invoice Date')
    axes[0].set_ylabel('Profit Margin')

    # Plot Quantity over Invoice Date
    axes[1].scatter(year_data['Invoice.Date'], year_data['Quantity'], color='green')
    axes[1].set_title(f'Quantity Over Date - Year {year}')
    axes[1].set_xlabel('Invoice Date')
    axes[1].set_ylabel('Quantity')

    plt.tight_layout()
    plt.show()


In [ ]:
unique_years = pilot_df['Year'].unique()

# Determine the number of plots needed
num_plots = len(unique_years)

# Calculate the number of rows and columns for the grid
num_rows = 2
num_cols = 3

# Create a figure and axes for the subplots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Flatten the axes array for easier indexing
axes = axes.flatten()

# Iterate over unique years and create plots
for i, year in enumerate(unique_years):
    # Filter data for the current year
    year_data = pilot_df[pilot_df['Year'] == year]

    # Plot Profit Margin vs Quantity for the current year
    axes[i].scatter(year_data['Profit_Margin'], year_data['Quantity'], color='blue')

    # Set labels and title for the current subplot
    axes[i].set_xlabel('Profit Margin')
    axes[i].set_ylabel('Quantity')
    axes[i].set_title(f'Profit Margin vs Quantity - Year {year}')

# Adjust layout and display the plots
plt.tight_layout()
plt.show()


### Transform Customer Category

In [ ]:
## Customer category
df['Customer.Category'] = df['Customer.Category'].fillna('Good')
mapping_customer_dict = {
    'P3': 'Good',
    'P1': 'Good',
    'P2': 'Good',
    #'Average Cost': 'Other',
    'P0': 'Good',
    #'OV': 'Other',
    #'List Price': 'Other',
}
# Remap values in specific columns
df['Customer.Category'] = df['Customer.Category'].replace(mapping_customer_dict)

### Inflation

In [ ]:
## Adding inflation
inflation = pd.read_excel('/content/drive/MyDrive/Capstone Data/inflation.xlsx',skiprows=11)
month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
inflation.rename(columns=month_map, inplace=True)
inflation.drop(columns=['HALF1','HALF2'],inplace=True)
melted_inflation = pd.melt(inflation, id_vars=['Year'], var_name='Month', value_name='Inflation')

### Region Mapping

In [ ]:
## Map Regions
state_to_region = {
    # Pacific
    'CA': 'Pacific', 'WA': 'Pacific', 'OR': 'Pacific', 'AK': 'Pacific', 'HI': 'Pacific',
    # West
    'MT':'West','ND':'West','ID': 'West', 'SD':'West', 'WY': 'West', 'NE':'West','NV':'West','UT': 'West','CO':'West','AZ':'West','NM':'West',
    # Midwest
    'IL': 'Midwest', 'OH': 'Midwest', 'MI': 'Midwest', 'IN': 'Midwest', 'WI': 'Midwest', 'IA': 'Midwest', 'MN': 'Midwest', 'MO': 'Midwest', 'KS': 'Midwest', 'NE': 'Midwest', 'SD': 'Midwest', 'ND': 'Midwest', 'KY':'Midwest', 'WV': 'Midwest',
    # Central
    'KS' : 'Central', 'MO': 'Central', 'TN': 'Central', 'OK' : 'Central', 'AR': 'Central',
    # Gulf Coast
    'TX': 'Gulf_Coast','LA': 'Gulf_Coast', 'MS': 'Gulf_Coast', 'AL': 'Gulf_Coast', 'GA': 'Gulf_Coast', 'FL': 'Gulf_Coast',
    # Mid-Atlantic
    'PA' : 'Mid-Atlantic',  'NJ': 'Mid-Atlantic', 'DE':'Mid-Atlantic', 'MD':'Mid-Atlantic', 'VA':'Mid-Atlantic', 'NC':'Mid-Atlantic', 'SC': 'Mid-Atlantic',
    # Northeast
    'NY': 'Northeast', 'CT': 'Northeast', 'MA': 'Northeast', 'RI': 'Northeast', 'VT': 'Northeast', 'NH': 'Northeast', 'ME': 'Northeast',
    # # Northeast
    # 'NY': 'Northeast', 'NJ': 'Northeast', 'PA': 'Northeast', 'CT': 'Northeast', 'MA': 'Northeast', 'RI': 'Northeast', 'VT': 'Northeast', 'NH': 'Northeast', 'ME': 'Northeast', 'MD': 'Northeast',
    # # Southeast
    # 'DE': 'Southeast', 'WV': 'Southeast', 'VA': 'Southeast', 'NC': 'Southeast', 'SC': 'Southeast', 'GA': 'Southeast', 'FL': 'Southeast', 'DC': 'Southeast', 'KY': 'Southeast','TN':'Southeast','AR': 'Southeast','MS':'Southeast','AL':'Southeast','LA':'Southeast',
    # # East
    # 'NY': 'East', 'NJ': 'East', 'PA': 'East', 'CT': 'East', 'MA': 'East', 'RI': 'East', 'VT': 'East', 'NH': 'East', 'ME': 'East', 'MD': 'East',
    # # Southeast
    # 'DE': 'East', 'WV': 'East', 'VA': 'East', 'NC': 'East', 'SC': 'East', 'GA': 'East', 'FL': 'East', 'DC': 'East', 'KY': 'East','TN':'East','AR': 'East','MS':'East','AL':'East','LA':'East',
    # # Southwest
    # 'TX': 'Southwest', 'AZ': 'Southwest', 'NM': 'Southwest', 'OK': 'Southwest',
    # # Other
    # 'BC':'Other','AB':'Other','QC':'Other'
}

df['Ship.To.State'] = df['Ship.To.State'].replace(state_to_region)

### Numerical Transformation

In [ ]:
numerical_columns = ['Inventory.Cost','Price','Quantity']
df[numerical_columns] = df[numerical_columns].replace({',': ''}, regex=True).astype(float)

### Margin

## Explore one specific product

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Capstone Data/filled_df.csv',low_memory=False)
df2 = pd.read_csv('/content/drive/MyDrive/Capstone Data/margin_final.csv', low_memory = False)

In [ ]:
df

In [ ]:
df2

In [ ]:
df['Per_Margin'] = (df['Price'] - df['Inventory.Cost']) / df['Inventory.Cost']

In [ ]:
df['Product.Classification'].value_counts()

In [ ]:
df = df[df['Customer.Category']=='Good']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error

categorical_features = ['Item.No.','Item.Group','Product.Classification']
numerical_features = ['Quantity','Inventory.Cost','Price']

# Create preprocessor
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', RandomForestRegressor())])

X = df[['Item.No.','Item.Group','Product.Classification','Quantity','Inventory.Cost']]
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

train_score = pipeline.score(X_train, y_train)
test_score = pipeline.score(X_test, y_test)

print(f"Train Score: {train_score}")
print(f"Test Score: {test_score}")


# rf = RandomForestRegressor(n_estimators = 300, max_features = 'sqrt', max_depth = 5, random_state = 42).fit(X_train, y_train)

# prediction = rf.predict(X_test)

# mse = mean_squared_error(y_test, prediction)
# rmse = mse**.5
# print(mse)
# print(rmse)

In [ ]:
test_df = df[(df['Customer.Category']=='Good')&(df['Item.Group']=='Const Supplies CS/SD')&(df['Product.Classification']=='SAFETY')&(df['Item.No.']=='GEC 990IK-L')]

In [ ]:
test_df.info()

In [ ]:
test_df['Ship.To.State'].value_counts()

In [ ]:
avg_price_quantity = test_df.groupby('Invoice.Date').agg({'Price': 'mean', 'Quantity': 'mean','Inventory.Cost':'mean','Per_Margin':'mean'}).reset_index()

avg_price_quantity.columns = ['Invoice.Date', 'Avg_Price', 'Avg_Quantity','Avg_Cost','Avg_Margin']

test_df = test_df.merge(avg_price_quantity, on='Invoice.Date', how='left')

In [ ]:
pilot_df = test_df[['Stock.Status','Avg_Price','Avg_Quantity','Avg_Cost', 'Avg_Margin','Invoice.Date','Year','Month','Quarter','Prev_Quarter']]

In [ ]:
pilot_df

In [ ]:
pilot_df.info()

In [ ]:
pilot_df[pilot_df['Year']==2016]

In [ ]:
years = pilot_df['Year'].unique()

# Plotting average price and average quantity separately for each year
for year in years:
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
    year_data = pilot_df[pilot_df['Year'] == year]
    year_data.plot(x='Invoice.Date', y='Avg_Price', ax=axes[0], title=f'Average Price Over Date - Year {year}', color='blue')
    # year_data.plot(x='Invoice.Date', y='Avg_Quantity', ax=axes[0], title=f'Average Price Over Date - Year {year}', color='green')
    year_data.plot(x='Invoice.Date', y='Avg_Quantity', ax=axes[1], title=f'Average Quantity Over Date - Year {year}', color='green')
    plt.tight_layout()
    plt.show()

In [ ]:
years = pilot_df['Year'].unique()

# Plotting average price and average quantity separately for each year
for year in years:
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
    year_data = pilot_df[pilot_df['Year'] == year]
    year_data.plot(x='Invoice.Date', y='Avg_Cost', ax=axes[0], title=f'Average Price Over Date - Year {year}', color='blue')
    year_data.plot(x='Invoice.Date', y='Avg_Margin', ax=axes[0], title=f'Average Price Over Date - Year {year}', color='blue')
    plt.show()

In [ ]:
test_df_2 = df[(df['Customer.Category']=='Good')&(df['Item.Group']=='Const Supplies CS/SD')&(df['Product.Classification']=='SAFETY')&(df['Item.No.']=='PBI 1500068')]

In [ ]:
avg_price_quantity = test_df_2.groupby('Invoice.Date').agg({'Price': 'mean', 'Quantity': 'mean','Inventory.Cost':'mean'}).reset_index()

avg_price_quantity.columns = ['Invoice.Date', 'Avg_Price', 'Avg_Quantity','Avg_Cost']

test_df_2 = test_df_2.merge(avg_price_quantity, on='Invoice.Date', how='left')

In [ ]:
years = test_df_2['Year'].unique()

# Plotting average price and average quantity separately for each year
for year in years:
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
    year_data = test_df_2[test_df_2['Year'] == year]
    year_data.plot(x='Invoice.Date', y='Avg_Price', ax=axes[0], title=f'Average Price Over Date - Year {year}', color='blue')
    year_data.plot(x='Invoice.Date', y='Avg_Quantity', ax=axes[1], title=f'Average Quantity Over Date - Year {year}', color='green')
    plt.tight_layout()
    plt.show()